# Script for visualizing Senegal map divided by departments. Global forest change dataset (Hansen 2019) is uploaded and in specific points treecover and loss can be visualized. The script can be implemented to visualize treecover and loss for specific region

#import libraries

In [2]:
import ee
import rasterio
import matplotlib.pyplot as plt
import os, sys
import numpy as np
import earthpy.plot as ep
import numpy.ma as ma
import collections
import pandas as pd
import seaborn
import pylab as pl
import geemap
import json
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

#Visualize map centered in Senegal

In [9]:
Map = geemap.Map(center=(14.782920, -15.274954), zoom=7)
Map

Map(center=[14.78292, -15.274954], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

#import forest cover change dataset selecting treecover, loss and lossyear band

In [10]:
gfc2019 = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')\
            .select(['treecover2000','loss','lossyear'])
            

In [5]:
#import shapefiles per Senegal departments and transform them to ee to format

In [11]:
senegal_shp = 'C:\\Users\\Caterina\\Desktop\\gadm36_SEN_shp\\gadm36_SEN_2.shp'#change with the correct path where the shapefiles are
department = geemap.shp_to_ee(senegal_shp)

In [12]:
#Diplay treecover and loss in the map (look up)

In [13]:
vis_params = {'bands': ['treecover2000', 'loss', 'lossyear'], 'min': 0.0, 'max': 20000.0, 'opacity': 1.0, 'gamma': 1.0}
Map.addLayer(gfc2014,vis_params,'Tree cover and loss')

#diplay departments in the map (look up)

In [14]:
json_data_2 = ee_to_geojson(department)
json_layer_2 = GeoJSON(data=json_data_2, name='Department JSON', hover_style={'fillColor': 'yellow' , 'fillOpacity': 0.5})
Map.add_layer(json_layer_2,{},'Department')

#make the map interactive: you can know the name of the department that you are looking at. In the plotting option you can visualize treecover and loss in specific point of the map

In [15]:
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl, GeoJSON 

html1 = HTML('''
    <h4>Country</h4>
    Go over an area
''')
html1.layout.margin = '0px 20px 20px 20px'
control1 = WidgetControl(widget=html1, position='bottomright')
Map.add_control(control1)

def update_html(feature, **kwargs):
    html1.value = '''
        <h4>Area code: <b>{}</b></h4>
        Area name: {}
    '''.format(feature['id'], feature['properties']['NAME_2'])

json_layer_2.on_hover(update_html)